## Preprocessing

In [3]:
pip install tensorflow

  Using cached tensorflow-2.6.2-cp36-cp36m-win_amd64.whl (423.3 MB)
  Using cached absl_py-0.15.0-py3-none-any.whl (132 kB)Note: you may need to restart the kernel to use updated packages.



  Using cached wrapt-1.12.1-py3-none-any.whl
  Using cached flatbuffers-1.12-py2.py3-none-any.whl (15 kB)
  Using cached h5py-3.1.0-cp36-cp36m-win_amd64.whl (2.7 MB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached clang-5.0-py3-none-any.whl
  Using cached tensorboard-2.6.0-py3-none-any.whl (5.6 MB)
  Using cached tensorflow_estimator-2.6.0-py2.py3-none-any.whl (462 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached keras-2.6.0-py2.py3-none-any.whl (1.3 MB)
  Using cached grpcio-1.44.0-cp36-cp36m-win_amd64.whl (3.4 MB)
  Using cached gast-0.4.0-py3-none-any.whl (9.8 kB)
  Using cached termcolor-1.1.0-py3-none-any.whl
  Using cached Keras_Preprocessing-1.1.2-py2.py3-none-any.whl (42 kB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Using cached cached_property-1.5.2-py2.py3-none-any.whl (7.6 kB)
  Using cached google_auth_oauthlib-0.4.6-py2.py3-none-any.whl (18 kB)


  Using cached Markdown-3.3.6-py3-none-any.whl (97 kB)


In [25]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [26]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns = ['EIN', 'NAME'])
application_df

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [27]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [28]:
# Look at APPLICATION_TYPE value counts for binning
app_type_values = application_df['APPLICATION_TYPE'].value_counts()
app_type_values

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T14        3
T25        3
T15        2
T29        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [29]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list(app_type_values[app_type_values < 500].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [30]:
# Look at CLASSIFICATION value counts for binning
classification_binning = application_df['CLASSIFICATION'].value_counts()
classification_binning

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C8210        1
C5200        1
C4200        1
C1732        1
C2190        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [31]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_values = classification_binning[classification_binning>1]
classification_values

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C2300       32
C7200       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1278       10
C1238       10
C1237        9
C1235        9
C7210        7
C1720        6
C4100        6
C2400        6
C1257        5
C1600        5
C0           3
C1260        3
C2710        3
C3200        2
C1267        2
C1246        2
C1234        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [32]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(classification_binning[classification_binning < 1000].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [34]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df = pd.get_dummies(application_df,dtype=float)
application_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [41]:
# Split our preprocessed data into our features and target arrays
y = application_df['IS_SUCCESSFUL'].values
X = application_df.drop(['IS_SUCCESSFUL'], axis=1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state = 42)

In [42]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [71]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn_model = tf.keras.models.Sequential()

# First hidden layer
nn_model.add(
    tf.keras.layers.Dense(units=80, input_dim=len(X_train[0]), activation="sigmoid"))

# Second hidden layer
nn_model.add(tf.keras.layers.Dense(units=30, activation="sigmoid"))

#3rd test adding 3rd hidden layer
nn_model.add(tf.keras.layers.Dense(units=7, activation="sigmoid"))

# Output layer
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))


# Check the structure of the model
nn_model.summary()


Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_33 (Dense)             (None, 80)                3520      
_________________________________________________________________
dense_34 (Dense)             (None, 30)                2430      
_________________________________________________________________
dense_35 (Dense)             (None, 7)                 217       
_________________________________________________________________
dense_36 (Dense)             (None, 1)                 8         
Total params: 6,175
Trainable params: 6,175
Non-trainable params: 0
_________________________________________________________________


In [72]:
# Compile the model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [73]:
# Train the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=200)

Epoch 1/200
804/804 [==============================] - 4s 4ms/step - loss: 0.6033 - accuracy: 0.6971
Epoch 2/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5754 - accuracy: 0.7284
Epoch 3/200
804/804 [==============================] - 3s 3ms/step - loss: 0.5726 - accuracy: 0.7292
Epoch 4/200
804/804 [==============================] - 3s 3ms/step - loss: 0.5669 - accuracy: 0.7282
Epoch 5/200
804/804 [==============================] - 3s 3ms/step - loss: 0.5608 - accuracy: 0.7297
Epoch 6/200
804/804 [==============================] - 3s 3ms/step - loss: 0.5578 - accuracy: 0.7297
Epoch 7/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5558 - accuracy: 0.7305
Epoch 8/200
804/804 [==============================] - 3s 3ms/step - loss: 0.5549 - accuracy: 0.7301
Epoch 9/200
804/804 [==============================] - 3s 3ms/step - loss: 0.5539 - accuracy: 0.7311
Epoch 10/200
804/804 [==============================] - 3s 3ms/step - loss: 0.5530 - accura

804/804 [==============================] - 3s 4ms/step - loss: 0.5366 - accuracy: 0.7371
Epoch 81/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5363 - accuracy: 0.7393
Epoch 82/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5365 - accuracy: 0.7381
Epoch 83/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5364 - accuracy: 0.7387
Epoch 84/200
804/804 [==============================] - 3s 3ms/step - loss: 0.5360 - accuracy: 0.7379
Epoch 85/200
804/804 [==============================] - 3s 3ms/step - loss: 0.5363 - accuracy: 0.7385
Epoch 86/200
804/804 [==============================] - 3s 3ms/step - loss: 0.5361 - accuracy: 0.7389
Epoch 87/200
804/804 [==============================] - 3s 3ms/step - loss: 0.5361 - accuracy: 0.7381
Epoch 88/200
804/804 [==============================] - 3s 3ms/step - loss: 0.5359 - accuracy: 0.7386
Epoch 89/200
804/804 [==============================] - 3s 3ms/step - loss: 0.5359 - accuracy: 

804/804 [==============================] - 3s 4ms/step - loss: 0.5317 - accuracy: 0.7413
Epoch 160/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5321 - accuracy: 0.7405
Epoch 161/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5315 - accuracy: 0.7413
Epoch 162/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5316 - accuracy: 0.7414
Epoch 163/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5316 - accuracy: 0.7411
Epoch 164/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5316 - accuracy: 0.7403
Epoch 165/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5312 - accuracy: 0.7414
Epoch 166/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5314 - accuracy: 0.7410
Epoch 167/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5313 - accuracy: 0.7403
Epoch 168/200
804/804 [==============================] - 3s 3ms/step - loss: 0.5315 - a

In [74]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5665 - accuracy: 0.7258
Loss: 0.5664629340171814, Accuracy: 0.7258309125900269


In [52]:
# Export our model to HDF5 file
nn_model.save("AlphabetSoupCharity.h5")